## Building A ChatBot

How to design and implement an LLM-powered chatbot.This chatbot will be able to have a conversation and remember previous interactions

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Loading API Key
groq_api_key = os.getenv('GROQ_API_KEY')

In [5]:
# Loading model
from langchain_groq import ChatGroq
model=ChatGroq(groq_api_key=groq_api_key,model_name='openai/gpt-oss-20b')
model

d:\Chatbot_With_Message_History_Using_Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000255408238C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025525C786E0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage
# User is giving the message
model.invoke([HumanMessage(content="Hey hi, my name is Krish and I am chief AI engineer.")])
# Here we have get the response from the AI from the model

AIMessage(content='Hello Krish! 👋 It’s great to meet the chief AI engineer. How can I assist you today? Whether it’s brainstorming, debugging, or exploring new AI concepts, I’m here to help.', additional_kwargs={'reasoning_content': 'We should respond with a friendly greeting, mention that we are an AI assistant. The user says "Hey hi, my name is Krish and I am chief AI engineer." We should acknowledge. Maybe ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 86, 'total_tokens': 184, 'completion_time': 0.10438541, 'prompt_time': 0.004397059, 'queue_time': 0.043167361, 'total_time': 0.108782469}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8fa9509a-c94b-4b40-93ef-e4a9bf7663d4-0', usage_metadata={'input_tokens': 86, 'output_tokens': 98, 'total_tokens': 184})

In [11]:
# Creating the sequence of messages
from langchain_core.messages import AIMessage
messages = (
    [
        HumanMessage(content="Hey, my name is Krish and I am a chief AI engineer."),
        AIMessage(content="Hello, Krish. It's nice to meet you as a chief engineer. What kind of projects are you working on these days? I am always eager to learn about the exciting work being done in the field of AI."),
        HumanMessage(content="Hey, what's my name and what do I do?")
    ]
)

response = model.invoke(messages)
print(response)


content='You’re Krish, and you’re a Chief AI Engineer. In that role, you’re responsible for overseeing AI strategy, guiding teams on model development, ensuring ethical AI practices, and driving innovation in AI products and services.' additional_kwargs={'reasoning_content': 'The user says: "Hey, what\'s my name and what do I do?" They previously said: "Hey, my name is Krish and I am a chief AI engineer." So the assistant should answer: name is Krish, they are a chief AI engineer. Also maybe elaborate. Ensure no policy violation. It\'s straightforward.'} response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 151, 'total_tokens': 272, 'completion_time': 0.122062419, 'prompt_time': 0.007431785, 'queue_time': 0.042793915, 'total_time': 0.129494204}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--7d599c8d-411c-4372-88eb-9cac41f8fa39-0' usage_metadata={'input_t

# Message History (How can the model remember the past conversation History)
Message history is essential for maintaining context across different sessions and users. In real-world applications, multiple sessions may occur simultaneously, and each session must retain its own context. This is managed using session IDs and message history.


In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
# Implementing Session-Based Message History
# We create a function to manage session-based message history. Each session ID corresponds to a separate chat history, stored in a dictionary.
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory :
    # Session id is use to distinguish between the differenrt sessions
    if session_id not in store :
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [18]:
config = {"configurable":{"session_id":"chat1"}}

In [20]:
response = with_message_history.invoke([HumanMessage(content="Hi, my name is Krish and I am an AI engineer.")], config=config)
print(response.content)

Nice to meet you again, Krish! How can I help you today? Whether you’re working on a new model, need code snippets, or just want to brainstorm ideas, feel free to let me know.


In [22]:
response1 = with_message_history.invoke([HumanMessage(content="What's is my name")],config=config)
print(response1.content)

Your name is Krish.


In [23]:
## Change the config
config1 = {"configurable":{"session_id":"chat2"}}


In [25]:
response1 = with_message_history.invoke(
    [HumanMessage(content="What's my name")
],config=config1)
print(response1.content)

I don’t have that information unless you share it with me. What would you like me to call you?


In [26]:
response2 = with_message_history.invoke([HumanMessage(content="My name is John")],config=config1)
print(response2.content)

Nice to meet you, John! How can I help you today?


In [28]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What's my name")
],config=config1)
print(response2.content)

Your name is John.


# Prompt Templates
Prompt templates help turn raw user information into a format that a language model can work with. If the raw user input is just a message, we can use prompt templates to structure it before passing it to the language model.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your availability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [31]:
# Invoking the chain
chain.invoke({"messages":[HumanMessage(content="Hii My name is Krish")]})

AIMessage(content='Hello Krish! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "Hii My name is Krish". They didn\'t ask a question. We can respond with a friendly greeting, maybe ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 97, 'total_tokens': 153, 'completion_time': 0.055791694, 'prompt_time': 0.004706613, 'queue_time': 0.042591473, 'total_time': 0.060498307}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a007abb-3199-4abe-a0b6-2bf8020a50f2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 56, 'total_tokens': 153})

In [32]:
config3={"configurable":{"session_id":"chat3"}}

In [35]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
config={"configurations":{"session_id":"chat3"}}
response =  with_message_history.invoke([HumanMessage(content="Hii My name is Krish")],config=config3)
print(response.content)

Hello Krish! 👋 How can I help you today?


In [36]:
## Multile Input (Adding more complexity)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to your ability in this language: {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = chat_prompt|model
response = chain.invoke({"messages":[HumanMessage(content="Hii My name is Krish")],"language":"Hindi"})
print(response.content)

नमस्ते, कृष्ण जी! आप कैसे हैं? मैं आपकी किस प्रकार सहायता कर सकता हूँ?


# Managing Chat History with Multiple Keys
When using multiple input variables, specify the correct input message key in the message history class. This ensures the chat history is saved and retrieved correctly.

In [39]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [40]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hii,I am Krish")],"language":"Hindi"},
    config=config
)
print(response.content)

नमस्ते, कृष्ण जी! आपसे मिलकर खुशी हुई। मैं आपकी किस प्रकार सहायता कर सकता हूँ?


In [43]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")],"language":"Hindi"},
    config=config
)
print(response.content)

आपका नाम कृष्ण है।


In [66]:
prompt=ChatPromptTemplate.from_messages([
    ("system","You are the language translator and your  work is to translate the input text from {language1} to {language2}"),
    MessagesPlaceholder(variable_name="messages")
])

In [67]:
language_translator_chain = prompt|model

In [ ]:
with_message_history = RunnableWithMessageHistory(
    language_translator_chain,
    get_session_history,
    input_messages_key="messages"
)

In [72]:
config5={"configurable":{"session_id":"chat5"}}

In [75]:
response = with_message_history.invoke(
    {
        "messages":[HumanMessage(content="What are you doing ?")], # Input Given by user
        "language1":"English",
        "language2":"Hindi"
    },
    config=config5
)

In [76]:
print(response.content)

आप क्या कर रहे हैं?


# Manage the Conversational History

The focus is on managing the conversation history when building a chatbot using Langchain. Proper management of conversation history is crucial, as an unmanaged list of messages can grow unbounded and potentially overflow the context window of the language model (LM). Therefore, it is important to add a step that limits the size of the messages being passed.

# Trim_Message

The trim_messages helper is used to reduce the number of messages sent to the model. It allows specifying how many tokens to keep and provides other parameters such as whether to always keep the system message and whether to allow partial messages.

The system message guides the AI’s behavior.

The human message provides input.

The AI message records output.

In [10]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

# Set of conversation between Human and AI
messages = [
    SystemMessage(content="You are a good assistant."),
    HumanMessage(content="Hey, I am Bob."),
    AIMessage(content="Hi."),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Nice."),
    HumanMessage(content="What is two plus two?"),
    AIMessage(content="Four."),
    HumanMessage(content="Thanks.")
]

trimmed = trimmer.invoke(messages)
trimmed

[SystemMessage(content='You are a good assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hey, I am Bob.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is two plus two?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Four.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks.', additional_kwargs={}, response_metadata={})]

In [15]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your availability in {language} language"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [16]:
# How to pass the trimmer in chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response = chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What icecream do i like")],
    "language":"English"
    }
)

print(response.content)

You mentioned that you like vanilla ice cream.


In [18]:
response1 = chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What math problem did I ask?")],
    "language":"English"
    }
)
print(response1.content)

You asked: “What is two plus two?”


In [19]:
response2 = chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What is the answer of two plus two")],
    "language":"English"
    }
)
print(response2.content)

Four.


In [26]:
## Let's wrap in the Messsage History
with_message_history =  RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config5={"configurable":{"session_id":"chat5"}}

In [ ]:
response2 = chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What is the answer of two plus two")], # Here we are define the old messaged with the new messages
    "language":"English"
    },
    config=config5
)
print(response2.content)

The answer is 4.
